In [1]:
import pickle
from util.params import Params
from util.env import set_param
from main import Main
import torch
from models.GDN import GDN
from test_loop import test
from train_loop import train


In [2]:
path = "./snapshot/gdn_swat/25_03_28_09_47_29/0/"

In [3]:
param: Params = pickle.load(file=open(f"{path}param.pickle", "rb"))
set_param(param)
print(param.task)

Tasks.next_sensors


In [4]:
main = Main(param=param)

# DATASET 

*Datasets.swat*
sensors count:  31
actuators count:  9
consts count:  12
['FIT101', 'LIT101', 'MV101', 'AIT201', 'AIT202', 'AIT203', 'FIT201', 'MV201', 'DPIT301', 'FIT301', 'LIT301', 'MV301', 'MV302', 'MV303', 'MV304', 'AIT401', 'AIT402', 'FIT401', 'LIT401', 'AIT501', 'AIT502', 'AIT503', 'AIT504', 'FIT501', 'FIT502', 'FIT503', 'FIT504', 'PIT501', 'PIT502', 'PIT503', 'FIT601']


In [5]:
train(main.model, main.val_dataloader,main.val_dataloader)

epoc 1 / 50: 100%|██████████| 387/387 [00:08<00:00, 44.67it/s, loss=0.589]
c:\Users\moji\miniconda3\envs\cuda_12_4\Lib\site-packages\torch\nn\_reduction.py:51: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


samples:  49499 49499 387


KeyboardInterrupt: 

In [ ]:

val_avg_loss, val_result = test(main.model, main.val_dataloader)

In [ ]:
test_avg_loss, test_result = test(
            main.model, main.test_dataloader, threshold=val_avg_loss
        )

In [ ]:
main_scores = main.get_score(test_result,val_result)

In [ ]:
scores=main.run()
scores